In [18]:
import pandas as pd
import numpy as np

In [19]:
df = pd.read_csv("spotify_fixed_nation_clean.csv")
df

,Unnamed: 0,Position,title,artist,Streams,region,start_date,end_date,country
0,0,1.0,Starboy,The Weeknd,167258.0,pt,2016-12-30,2017-01-06,CA
1,1,2.0,Say You Won't Let Go,James Arthur,113379.0,pt,2016-12-30,2017-01-06,GB
2,2,3.0,Closer,The Chainsmokers,113157.0,pt,2016-12-30,2017-01-06,US
3,3,4.0,I Feel It Coming,The Weeknd,111042.0,pt,2016-12-30,2017-01-06,CA
4,4,5.0,Let Me Love You,DJ Snake,109443.0,pt,2016-12-30,2017-01-06,FR
...,...,...,...,...,...,...,...,...,...
42851,42851,196.0,Rainha da Favela,Ludmilla,39097.0,pt,2020-12-18,2020-12-25,BR
42852,42852,197.0,Sólido,Instinto 26,38983.0,pt,2020-12-18,2020-12-25,PT
42853,42853,198.0,Tranquilo,Aragão,38804.0,pt,2020-12-18,2020-12-25,PT
42854,42854,199.0,Reminds Me Of You,Juice WRLD,38487.0,pt,2020-12-18,2020-12-25,US


In [22]:
#Streams and count by artist

streams_by_artist = pd.pivot_table(df, values="Streams",index=['artist', 'country'], aggfunc={"Streams":np.sum, "artist":"count"})
streams_by_artist = streams_by_artist.rename(columns={"artist": "Frequência", "Streams": "Audições"})
streams_by_artist = streams_by_artist.reset_index()
streams_by_artist = streams_by_artist.rename(columns={"artist": "Artista", "country": "País"})
#streams_by_artist.sort_values(by=["Streams"],ascending=False)

streams_by_artist.to_csv("top_streams.csv")
streams_by_artist

,Artista,País,Audições,Frequência
0,21 Savage,US,3163440.0,82
1,24kGoldn,US,4347777.0,30
2,5 Seconds of Summer,AU,6203782.0,117
3,6LACK,US,270935.0,11
4,6ix9ine,US,5356523.0,108
...,...,...,...,...
728,benny blanco,US,5963695.0,104
729,blackbear,US,2304704.0,64
730,gnash,US,443357.0,16
731,mishlawi,PT,10141404.0,220


In [25]:
streams_by_artist.loc[streams_by_artist['País'] == 'AN', 'País'] = "AO"
pt = streams_by_artist.loc[streams_by_artist['País'] == 'PT']
non_pt = streams_by_artist.loc[streams_by_artist['País'] != 'PT']
lusophone = streams_by_artist.loc[(streams_by_artist['País'] == 'PT') | (streams_by_artist['País'] == 'BR') | (streams_by_artist['País'] == 'AO') | (streams_by_artist['País'] == 'PT/AN') | (streams_by_artist['País'] == 'CV')]

print(pt.count()/len(streams_by_artist))
print(non_pt.count()/len(streams_by_artist))
print(lusophone.count()/len(streams_by_artist))

Artista       0.115962
País          0.115962
Audições      0.115962
Frequência    0.115962
dtype: float64
Artista       0.884038
País          0.884038
Audições      0.884038
Frequência    0.884038
dtype: float64
Artista       0.223738
País          0.223738
Audições      0.223738
Frequência    0.223738
dtype: float64


In [5]:
lusophone_countries = ["PT", "BR", "AO", "CV", "PT/AN"]

def share_pt(x):
        return len([i for i in x if i == 'PT'])/len(x)

def share_lusophone(x):
        return len([i for i in x if i in lusophone_countries])/len(x)

def share_foreign(x):
        return len([i for i in x if i != 'PT'])/len(x)

In [87]:
#racios por numero de entradas

df['year'] = pd.DatetimeIndex(df['start_date']).year
df['month'] = pd.DatetimeIndex(df['start_date']).month

ratios = df.groupby(['year', "month"]).agg({'country' : share_pt})
ratios["Lusófona"] = df.groupby(['year', "month"]).agg({'country' : share_lusophone})
ratios["Estrangeira"] = df.groupby(['year', "month"]).agg({'country' : share_foreign})
ratios = ratios.reset_index()
ratios = ratios.rename(columns={"country": "Portuguesa", "month": "MONTH", "year":"YEAR"})

#cols = list(ratios.columns.values)

#cleaning the table for chart

ratios["Data"] = pd.to_datetime(ratios.YEAR.astype(str) + '/' + ratios.MONTH.astype(str))
ratios["Data"] = pd.to_datetime(ratios["Data"]).dt.date
ratios = ratios[['Data','Portuguesa', 'Lusófona', 'Estrangeira']]
ratios['Portuguesa'] = ratios['Portuguesa'] * 100
ratios['Lusófona'] = ratios['Lusófona'] * 100
ratios['Estrangeira'] = ratios['Estrangeira'] * 100

#ratios = ratios.transpose()
ratios = ratios.reset_index()

#ratios.to_csv("ratios_pt_luso.csv")

In [39]:
nov_2019 = df.loc[(df["year"] ==2019) & (df["month"] == 11)].groupby(['artist'], sort=False)["Streams"].agg(np.sum)
nov_2019.head(30)

artist
Julinho Ksd        4697709.0
Bispo              1678337.0
Tones And I        1317106.0
Travis Scott       1533787.0
Nenny              1102893.0
Wet Bed Gang       2242708.0
MC Du Black         768183.0
Selena Gomez        980999.0
Plutónio           3983124.0
Pk                  739413.0
Post Malone        2097406.0
Phoenix Rdc         991982.0
Piruka             1261225.0
Lil Tecca           973979.0
Shawn Mendes        706834.0
Deejay Telio       1350835.0
TAY                 814349.0
Sippinpurpp         495388.0
Lewis Capaldi       717276.0
Dua Lipa            555335.0
Ed Sheeran         1175863.0
Ali Gatie           526101.0
Y2K                 492434.0
Billie Eilish      1617347.0
Maroon 5            557220.0
MEDUZA              733600.0
ProfJam            1121935.0
Lil Nas X           592745.0
Slow J             1841162.0
Fernando Daniel     980584.0
Name: Streams, dtype: float64

In [86]:
df['year'] = pd.DatetimeIndex(df['start_date']).year
df['month'] = pd.DatetimeIndex(df['start_date']).month

ratios_streams = df.loc[(df["country"] =="PT")].groupby(['year', "month"]).agg({'Streams' : np.sum})
ratios_streams["Lusófona"] = df.loc[(df['country'] == 'PT') | (df['country'] == 'BR') | (df['country'] == 'AO') | (df['country'] == 'PT/AN') | (df['country'] == 'CV')].groupby(['year', "month"]).agg({'Streams' : np.sum})
ratios_streams["Estrangeira"] = df.loc[(df["country"] !="PT")].groupby(['year', "month"]).agg({'Streams' : np.sum})

ratios_streams = ratios_streams.reset_index()
ratios_streams = ratios_streams.rename(columns={"Streams": "Portuguesa", "month": "MONTH", "year":"YEAR"})

#cols = list(ratios_streams.columns.values)

#cleaning the table for chart

ratios_streams["Data"] = pd.to_datetime(ratios_streams.YEAR.astype(str) + '/' + ratios_streams.MONTH.astype(str))
ratios_streams["Data"] = pd.to_datetime(ratios_streams["Data"]).dt.date
ratios_streams = ratios_streams[['Data','Portuguesa', 'Lusófona', 'Estrangeira']]

ratios_streams = ratios_streams.reset_index()

ratios_streams.to_csv("streams_by_category.csv", date_format = "%Y/%m")

ratios_streams_transposed = ratios_streams.transpose()
#ratios_streams_transposed.to_csv("streams_by_category_transposed.csv", date_format = "%Y/%m")

In [26]:
streams_by_artist

,Artista,País,Audições,Frequência
0,21 Savage,US,3163440.0,82
1,24kGoldn,US,4347777.0,30
2,5 Seconds of Summer,AU,6203782.0,117
3,6LACK,US,270935.0,11
4,6ix9ine,US,5356523.0,108
...,...,...,...,...
728,benny blanco,US,5963695.0,104
729,blackbear,US,2304704.0,64
730,gnash,US,443357.0,16
731,mishlawi,PT,10141404.0,220
